In [126]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential

##Importamos las matrices a utilizar

In [41]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [62]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

In [55]:
id= '1OrAkBep-bmOz-P-QR3P5csDejgiJG9bL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_test.csv')

id= '17bNeSA9q59vTO6OW2lBTJPyw29QGQSxf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_train.csv')

id= '1Ffgm1RoYdjqMuQgTap3o5bOSaNjIdO7l'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_doc2vec_solo_text.csv')

id= '1Q7m92Pbm08m0t9X8Q975IUkPWEoZQ--R'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_doc2vec_solo_text.csv')

id= '1qvK0vblQPKJSgE8dTphy7sactLSVp6xf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_doc2vec_text_keyword.csv')

id= '1bQDT6HcEy46m2j2n92OLi5NmufCqaZ5Z'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_doc2vec_text_keyword.csv')

In [56]:
matriz_test_doc2vec_solo_text = pd.read_csv('matriz_test_doc2vec_solo_text.csv')
matriz_train_doc2vec_solo_text = pd.read_csv('matriz_train_doc2vec_solo_text.csv')

matriz_test_doc2vec_text_keyword = pd.read_csv('matriz_test_doc2vec_text_keyword.csv')
matriz_train_doc2vec_text_keyword = pd.read_csv('matriz_train_doc2vec_text_keyword.csv')

doc2vec_features_numericos_test = pd.read_csv('doc2vec_features_numericos_test.csv')
doc2vec_features_numericos_train = pd.read_csv('doc2vec_features_numericos_train.csv')

##SPLIT DE LAS MATRICES

In [64]:
y = train.target

In [72]:
x_train_text, x_validation_text, y_train_text, y_validation_text = train_test_split(matriz_train_doc2vec_solo_text, y, test_size=0.3, stratify=y)
x_train_text_keyword, x_validation_text_keyword, y_train_text_keyword, y_validation_text_keyword = train_test_split(matriz_train_doc2vec_text_keyword, y, test_size=0.3, stratify=y)
x_train_numericos, x_validation_numericos, y_train_numericos, y_validation_numericos = train_test_split(doc2vec_features_numericos_train, y, test_size=0.3, stratify=y)

##KNN

Funcion que aplica KNN con GridSearch y CV para encontrar los mejores hiperparametros para cada matriz probada.

In [66]:
def knn_con_gridsearch(x_train, y_train, x_validation, y_validation):

  knn = KNeighborsClassifier()
  params_knn = {'n_neighbors': np.arange(1, 25)}
  knn_gs = GridSearchCV(knn, params_knn, cv=5)
  knn_gs.fit(x_train, y_train)
  knn_best = knn_gs.best_estimator_
  
  print(knn_gs.best_params_)
  print('knn score: {}'.format(knn_best.score(x_validation, y_validation)))
  
  return knn_best

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [67]:
knn_text = knn_con_gridsearch(x_train_text, y_train_text, x_validation_text, y_validation_text)

{'n_neighbors': 4}
knn score: 0.5586690017513135


In [68]:
knn_text_keyword = knn_con_gridsearch(idsearch(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

{'n_neighbors': 10}
knn score: 0.6055166374781086


In [69]:
knn_numericos = knn_con_gridsearch(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

{'n_neighbors': 24}
knn score: 0.6663747810858144


Realizamos la prediccion con el modelo que haya dado mayor score

In [ ]:
#prediction_knn = knn_best.predict(doc2vec_features_numericos_test)
#submission_knn = pd.DataFrame(data={'id':test['id'], 'target': prediction_knn})
#submission_knn.to_csv('doc2vec_knn.csv', index=False)

#RED NEURONAL PROFUNDA

In [70]:
def DNN_model(shape, nClasses, dropout=0.5):
    model = Sequential()
    node = 512
    nLayers = 4
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [75]:
model_DNN_text = DNN_model(matriz_train_doc2vec_solo_text.shape[1], 2)
model_DNN_text.fit(matriz_train_doc2vec_solo_text, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
 - 2s - loss: 0.6957 - accuracy: 0.5497
Epoch 2/20
 - 2s - loss: 0.6872 - accuracy: 0.5622
Epoch 3/20
 - 2s - loss: 0.6842 - accuracy: 0.5602
Epoch 4/20
 - 2s - loss: 0.6818 - accuracy: 0.5647
Epoch 5/20
 - 2s - loss: 0.6775 - accuracy: 0.5774
Epoch 6/20
 - 2s - loss: 0.6769 - accuracy: 0.5749
Epoch 7/20
 - 2s - loss: 0.6755 - accuracy: 0.5638
Epoch 8/20
 - 2s - loss: 0.6711 - accuracy: 0.5848
Epoch 9/20
 - 2s - loss: 0.6669 - accuracy: 0.5870
Epoch 10/20
 - 2s - loss: 0.6577 - accuracy: 0.5970
Epoch 11/20
 - 2s - loss: 0.6535 - accuracy: 0.6116
Epoch 12/20
 - 2s - loss: 0.6493 - accuracy: 0.6164
Epoch 13/20
 - 2s - loss: 0.6376 - accuracy: 0.6344
Epoch 14/20
 - 2s - loss: 0.6335 - accuracy: 0.6394
Epoch 15/20
 - 2s - loss: 0.6228 - accuracy: 0.6540
Epoch 16/20
 - 2s - loss: 0.6132 - accuracy: 0.6536
Epoch 17/20
 - 2s - loss: 0.6093 - accuracy: 0.6649
Epoch 18/20
 - 2s - loss: 0.5932 - accuracy: 0.6737
Epoch 19/20
 - 2s - loss: 0.5846 - accuracy: 0.6867
Epoch 20/20
 - 2s - l

In [77]:
model_DNN_text_keyword = DNN_model(matriz_train_doc2vec_text_keyword.shape[1], 2)
model_DNN_text_keyword.fit(matriz_train_doc2vec_text_keyword, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
 - 2s - loss: 0.6958 - accuracy: 0.5455
Epoch 2/20
 - 2s - loss: 0.6811 - accuracy: 0.5697
Epoch 3/20
 - 2s - loss: 0.6790 - accuracy: 0.5744
Epoch 4/20
 - 2s - loss: 0.6697 - accuracy: 0.5866
Epoch 5/20
 - 2s - loss: 0.6710 - accuracy: 0.5908
Epoch 6/20
 - 2s - loss: 0.6598 - accuracy: 0.6070
Epoch 7/20
 - 2s - loss: 0.6511 - accuracy: 0.6197
Epoch 8/20
 - 2s - loss: 0.6481 - accuracy: 0.6343
Epoch 9/20
 - 2s - loss: 0.6316 - accuracy: 0.6432
Epoch 10/20
 - 2s - loss: 0.6251 - accuracy: 0.6564
Epoch 11/20
 - 2s - loss: 0.6149 - accuracy: 0.6639
Epoch 12/20
 - 2s - loss: 0.5999 - accuracy: 0.6817
Epoch 13/20
 - 2s - loss: 0.5943 - accuracy: 0.6815
Epoch 14/20
 - 2s - loss: 0.5756 - accuracy: 0.7006
Epoch 15/20
 - 2s - loss: 0.5663 - accuracy: 0.7075
Epoch 16/20
 - 2s - loss: 0.5609 - accuracy: 0.7098
Epoch 17/20
 - 2s - loss: 0.5427 - accuracy: 0.7259
Epoch 18/20
 - 2s - loss: 0.5372 - accuracy: 0.7303
Epoch 19/20
 - 2s - loss: 0.5217 - accuracy: 0.7381
Epoch 20/20
 - 2s - l

In [78]:
model_DNN_text_keyword = DNN_model(doc2vec_features_numericos_train.shape[1], 2)
model_DNN_text_keyword.fit(doc2vec_features_numericos_train, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
 - 2s - loss: 0.9726 - accuracy: 0.5273
Epoch 2/20
 - 2s - loss: 0.7036 - accuracy: 0.5366
Epoch 3/20
 - 2s - loss: 0.6893 - accuracy: 0.5577
Epoch 4/20
 - 2s - loss: 0.6757 - accuracy: 0.5749
Epoch 5/20
 - 2s - loss: 0.6631 - accuracy: 0.5914
Epoch 6/20
 - 2s - loss: 0.6603 - accuracy: 0.6058
Epoch 7/20
 - 2s - loss: 0.6497 - accuracy: 0.6233
Epoch 8/20
 - 2s - loss: 0.6434 - accuracy: 0.6291
Epoch 9/20
 - 2s - loss: 0.6449 - accuracy: 0.6251
Epoch 10/20
 - 2s - loss: 0.6411 - accuracy: 0.6368
Epoch 11/20
 - 2s - loss: 0.6363 - accuracy: 0.6438
Epoch 12/20
 - 2s - loss: 0.6312 - accuracy: 0.6461
Epoch 13/20
 - 2s - loss: 0.6286 - accuracy: 0.6607
Epoch 14/20
 - 2s - loss: 0.6278 - accuracy: 0.6538
Epoch 15/20
 - 2s - loss: 0.6264 - accuracy: 0.6578
Epoch 16/20
 - 2s - loss: 0.6268 - accuracy: 0.6624
Epoch 17/20
 - 2s - loss: 0.6236 - accuracy: 0.6603
Epoch 18/20
 - 2s - loss: 0.6221 - accuracy: 0.6654
Epoch 19/20
 - 2s - loss: 0.6214 - accuracy: 0.6635
Epoch 20/20
 - 2s - l

In [ ]:
#prediction_dnn = model_DNN.predict_classes(matriz_test_doc2vec_text_keyword)
#submission_dnn = pd.DataFrame(data={'id':test['id'], 'target': prediction_dnn})
#submission_dnn.to_csv('doc2vec_dnn.csv', index=False)

#RANDOM FOREST

Funcion que aplica RANDOM FOREST con GridSearch y CV para encontrar los mejores hiperparametros para cada matriz probada.

In [79]:
def RandomForest_con_gridsearch(x_train, y_train, x_validation, y_validation):

  rf = RandomForestClassifier()
  params_rf = {'n_estimators': [50, 100, 200]}
  rf_gs = GridSearchCV(rf, params_rf, cv=5)
  rf_gs.fit(x_train, y_train)
  rf_best = rf_gs.best_estimator_

  print(rf_gs.best_params_)
  print('rf: {}'.format(rf_best.score(x_validation, y_validation)))

  return rf_best

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [80]:
rf_text = RandomForest_con_gridsearch(x_train_text, y_train_text, x_validation_text, y_validation_text)

{'n_estimators': 100}
rf: 0.5547285464098074


In [81]:
rf_text_keyword = RandomForest_con_gridsearch(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

{'n_estimators': 200}
rf: 0.5998248686514887


In [82]:
rf_numericos = RandomForest_con_gridsearch(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

{'n_estimators': 200}
rf: 0.6549912434325744


In [ ]:
#prediction_rf_text = rf_text.predict(doc2vec_features_numericos_test)
#submission_rf = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf})
#submission_rf.to_csv('doc2vec_rf.csv', index=False)

###BAGGING

In [108]:
seed = 7
num_trees = 100

BAGGING CLASSIFIER

In [109]:
def baggingClassifier(x_train, y_train, x_validation, y_validation):

  kfold = model_selection.KFold(n_splits=10)
  cart = DecisionTreeClassifier()
  model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
  results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)

  print('Bagging classifier: {}'.format(results.mean()))

  return model

In [111]:
bagging_text = baggingClassifier(x_train_text, y_train_text, x_validation_text, y_validation_text)

Bagging classifier: 0.5554824947144418


In [112]:
bagging_text_keyword = baggingClassifier(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

Bagging classifier: 0.5554824947144418


In [113]:
bagging_numericos = baggingClassifier(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

Bagging classifier: 0.5554824947144418


In [ ]:
#prediction_bagg = model.predict(doc2vec_features_numericos_test)
#submission_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_bagg})
#submission_bagg.to_csv('doc2vec_bagg.csv', index=False)

BAGGING CON RANDOM FOREST

In [114]:
def baggingRandomForest(x_train, y_train, x_validation, y_validation):

  max_features = 3
  kfold = model_selection.KFold(n_splits=10)
  model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
  results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)

  print('Bagging Random Forest: {}'.format(results.mean()))

  return model

In [116]:
baggin_rf_text = baggingRandomForest(x_train_text, y_train_text, x_validation_text, y_validation_text)

Bagging Random Forest: 0.5532482470571598


In [117]:
baggin_rf_text_keyword = baggingRandomForest(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

Bagging Random Forest: 0.5632332095150394


In [118]:
baggin_rf_numericos = baggingRandomForest(x_train_numericos,y_train_numericos, x_validation_numericos, y_validation_numericos)

Bagging Random Forest: 0.5570581601084358


In [ ]:
#prediction_rf_bagg = model.predict(matriz_test_doc2vec_text_keyword)
#submission_rf_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf_bagg})
#submission_rf_bagg.to_csv('doc2vec_rf_bagg.csv', index=False)

#NuSVC

In [136]:
def nusvc(x_train, y_train):

  nusvc = NuSVC(probability=True)
  grid_nusvc = GridSearchCV(nusvc, cv=StratifiedKFold(n_splits=5), param_grid={'nu': [0.4, 0.5]}, scoring=['f1_macro'], refit=False)
  grid_nusvc.fit(x_train, y_train)

  print('nusvc: {}'.format(grid_nusvc.cv_results_['mean_test_f1_macro'][0]))

  return grid_nusvc

In [142]:
nusvc_text = nusvc(x_train_text, y_train_text)

nusvc: 0.5476285040781657


In [138]:
nusvc_text_keyword = nusvc(x_train_text_keyword, y_train_text_keyword)

nusvc: 0.5845603562957474


In [140]:
nusvc_numericos = nusvc(x_train_numericos,y_train_numericos)

nusvc: 0.6094480831000061


In [ ]:
#prediction_nuscv = nuscv.predict(doc2vec_features_numericos_test)
#submission_nuscv = pd.DataFrame(data={'id':test['id'], 'target': prediction_nuscv})
#submission_nuscv.to_csv('doc2vec_nuscv.csv', index=False)

#XGBoost

In [104]:
def xgboost(x_train, y_train, x_validation, y_validation):

  model = XGBClassifier()
  model.fit(x_train, y_train)
  y_pred = model.predict(x_validation)
  predictions = [round(value) for value in y_pred]
  accuracy = accuracy_score(y_validation, predictions)

  print("Accuracy xgboost: %.5f%%" % accuracy)

  return model

In [105]:
xgboost_text = xgboost(x_train_text, y_train_text, x_validation_text, y_validation_text)

Accuracy xgboost: 0.56305%


In [106]:
xgboost_text_keyword = xgboost(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

Accuracy xgboost: 0.59238%


In [107]:
xgboost_numericos = xgboost(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

Accuracy xgboost: 0.66200%


In [ ]:
#submission_xgboost = pd.DataFrame(data={'id':test['id'], 'target': prediction_xgboost})
#submission_xgboost.to_csv('tf_idf_xgboost.csv', index=False)